In [ ]:
import csv
import os
import numpy as np
from sklearn import tree                       
from osgeo import gdal
import ospybook as pb
folder = r'D:\Projects\VsCode\Python\img_processing_system\osgeopy-data\Landsat\Utah'
raster_fns = ['LE70380322000181EDC02_60m.tif', 'LE70380322000181EDC02_TIR_60m.tif']
out_fn = 'tree_prediction60.tif'
train_fn = r'D:\Projects\VsCode\Python\img_processing_system\osgeopy-data\Utah\training_data.csv'
gap_fn = r'D:\Projects\VsCode\Python\img_processing_system\osgeopy-data\Utah\landcover60.tif'
os.chdir(folder)


# Read coordinates and class from csv
xys = []                                                                 
classes = []                                                            
with open(train_fn) as fp:                                             
    reader = csv.reader(fp)                                            
    next(reader)                                                       
    for row in reader:      
        print(row)                                            
        xys.append([float(n) for n in row[:2]])                         
        classes.append(int(row[2]))    

In [ ]:
# Get pixel offsets
#   Then you open one of the raster datasets so you can create a transformer object to
#   convert between map coordinates and pixel offsets. You use this with your list of coordinates
#   to get pixel offsets in two lists called cols and rows.
# 问题：没理解pixel offsets 代表什么？
ds = gdal.Open(raster_fns[0])                                          
pixel_trans = gdal.Transformer(ds, None, [])                           
offset, ok = pixel_trans.TransformPoints(True, xys) # xys是从training_data.csv中取的。
cols, rows, z = zip(*offset)  

In [ ]:
print(pixel_trans)
print("offset: ", offset)
print("ok: ", ok)
print("cols: ", cols)
print("rows: ", rows)
print("z: ", z)

In [ ]:
data = pb.stack_bands(raster_fns)

# Convert rows and cols to integer numpy arrays for indexing
rows_int = np.array(rows).astype(int)
cols_int = np.array(cols).astype(int)

# Sample satellite data at pixel offsets
sample = data[rows_int, cols_int, :]

In [ ]:
# Fit the classification tree
clf = tree.DecisionTreeClassifier(max_depth=5)                           
clf = clf.fit(sample, classes)

# Apply model to satellite data
rows, cols, bands = data.shape
data2d = np.reshape(data, (rows * cols, bands)) 
prediction = clf.predict(data2d) 
prediction = np.reshape(prediction, (rows, cols))

# Set pixels with no satellite data to 0
prediction[np.sum(data, 2) == 0] = 0   

predict_ds = pb.make_raster(ds, out_fn, prediction, gdal.GDT_Byte, 0)
predict_ds.FlushCache()
levels = pb.compute_overview_levels(predict_ds.GetRasterBand(1))
predict_ds.BuildOverviews('NEAREST', levels)

# Copy colormap from SWReGAP dataset
gap_ds = gdal.Open(gap_fn)                                             
colors = gap_ds.GetRasterBand(1).GetRasterColorTable()                 
predict_ds.GetRasterBand(1).SetRasterColorTable(colors) 

del ds